In [1]:
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

We are going to use pyteseract to extract min and max temps so we have the scale range.

In [2]:
def run_ocr_on_coords(image_path):
    # Hardcoded coordinates for min/max temp regions
    coords = {
        "min_top_left": {"x": 36, "y": 31, "width": 34, "height": 15},
        "max_top_left": {"x": 35, "y": 45, "width": 35, "height": 17},
        "max_right": {"x": 197, "y": 38, "width": 38, "height": 18},
        "min_right": {"x": 199, "y": 261, "width": 36, "height": 19}
    }

        # Load the image
    img = Image.open(image_path)

    # This dictionary will store for each region both the OCR text and its confidence
    results = {}

    for label, coord in coords.items():
        # Extract coordinates
        x, y, w, h = coord["x"], coord["y"], coord["width"], coord["height"]
        cropped = img.crop((x, y, x + w, y + h))
        
        # Preprocess: grayscale, contrast enhancement, and sharpening
        gray = ImageOps.grayscale(cropped)
        enhanced = ImageEnhance.Contrast(gray).enhance(2.0)
        sharp = enhanced.filter(ImageFilter.SHARPEN)
        
        # Use image_to_data to get detailed OCR results including confidence values
        data = pytesseract.image_to_data(
            sharp,
            output_type=pytesseract.Output.DICT,
            config='--psm 6 -c tessedit_char_whitelist=0123456789.'
        )
        
        best_text = ""
        best_conf = -1  # Initialize with -1 to handle non-detections
        # Iterate through detected words (or fragments)
        for i in range(len(data['text'])):
            text_candidate = data['text'][i].strip()
            try:
                # Confidence returned as string; convert it to a float
                conf = float(data['conf'][i])
            except ValueError:
                conf = -1  # or simply skip if conversion fails
            # Pick the candidate with the highest confidence
            if text_candidate and conf > best_conf:
                best_text = text_candidate
                best_conf = conf

        results[label] = {"value": best_text, "conf": best_conf}

    # For regions where the same value is expected (duplicates), select the one with higher confidence.
    def choose_better(region1, region2):
        r1 = results.get(region1)
        r2 = results.get(region2)
        if r1 and r2:
            return r1 if r1["conf"] >= r2["conf"] else r2
        return r1 or r2

    final_min = choose_better("min_top_left", "min_right")
    final_max = choose_better("max_top_left", "max_right")

    # Return final chosen values and associated confidence scores.
    return {
        "min": final_min["value"], "min_conf": final_min["conf"],
        "max": final_max["value"], "max_conf": final_max["conf"]
    }




# Example usage
if __name__ == "__main__":
    image_path = "thermal_imges_for_exp\CH01.jpeg"  # Replace with your path
    results = run_ocr_on_coords(image_path)
    for key, val in results.items():
        print(f"{key}: {val}")


min: 49.9
min_conf: 70.0
max: 71.8
max_conf: 85.0
